## Exploración interactiva de remolinos ciclónicos y anticlónicos
Este notebook nos permite explorar interactivamente los remolinos del producto de AVISO [Global mesoscale eddy trajectory product]([https://www.aviso.altimetry.fr/en/data/products/value-added-products/global-mesoscale-eddy-trajectory-product.html) y los perfiles de [Argo](https://argo.ucsd.edu/) más cercanos a estos remolinos en tiempo (dentro de una ventana de 15 días) y espacio (dentro de hasta 0.25 grados de distancia al contorno inicial del remolino). Usamos [Argopy](https://argopy.readthedocs.io/en/latest/index.html) para obtener estos perfiles.
1. Selecciona si deseas explorar los remolinos ciclónicos o anticlónicos.
2. Elige un remolino específico (número entre 1-13537 para ciclónicos y 1-13569 para anticiclónicos)
3. Carga los datos para ver sus características (toma un poco de tiempo, ya que la base de datos es grande).  
4. Visualiza la ubicación y forma del remolino en un mapa y/o los perfiles de Argo más cercanos al remolino. 

In [ ]:
import argopy as argo
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
sns.set_context("talk")

#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [ ]:
def load_eddy_data(eddy_type):
    # Load the appropriate eddy dataset based on the type
    if eddy_type == 'C':
        filepath = 'eddy_traj_nrt_3.2exp_cyclonic_20180101_20180610.nc'
    elif eddy_type == 'A':
        filepath = 'eddy_traj_nrt_3.2exp_anticyclonic_20180101_20180604.nc'
    else:
        raise ValueError("el tipo de remolino debe ser  'C' (ciclónico) o 'A' (anticiclónico)" )

    eddy_ds = xr.open_dataset(filepath)  
    return eddy_ds

def extract_eddy(eddy_ds, eddy_num):
    # Extract the latitude and longitude of the effective contour for the specified eddy number
    eddy_sel = eddy_ds.where(eddy_ds.track == eddy_num, drop=True)
    lat = eddy_sel.effective_contour_latitude.data[0,:]  
    lon = eddy_sel.effective_contour_longitude.data[0,:]
    lat_center = eddy_sel.latitude.data[0]
    lon_center = eddy_sel.longitude.data[0]
    lat_last = eddy_sel.effective_contour_latitude.data[-1,:]  
    lon_last = eddy_sel.effective_contour_longitude.data[-1,:]
    lat_center_last = eddy_sel.latitude.data[-1]
    lon_center_last = eddy_sel.longitude.data[-1]
    eddy_time = pd.to_datetime(eddy_sel.time[0].values)
    eddy_sel_time_ini = pd.to_datetime(eddy_sel.time[0].values)
    eddy_sel_time_end = pd.to_datetime(eddy_sel.time[-1].values)
    eddy_radius = eddy_sel.effective_radius.data[0]
    return (lat, lon, lat_center, lon_center, eddy_time, eddy_radius, 
            eddy_sel_time_ini, eddy_sel_time_end, eddy_sel, lat_last, 
            lon_last, lat_center_last, lon_center_last)

def plot_eddy_map(lat, lon, lat_center, lon_center, eddy_time, eddy_sel,  
                 eddy_type_str, eddy_num, eddy_sel_time_ini, eddy_sel_time_end): 
    fig = plt.figure(figsize=(10, 10))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines()
    ax.set_global()
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.OCEAN)
    ax.gridlines(draw_labels=True)
    col='blue' if eddy_type_str == 'ciclónico' else 'red'
    ax.plot(lon, lat, marker='o', markersize=2, linestyle='-', color=col,
            label=f' Contorno inicial', transform=ccrs.PlateCarree())
    ax.plot(lon_center, lat_center, marker='x', markersize=10, color=col, 
            label=f'Centro inicial', transform=ccrs.PlateCarree())
    # inset Axes...
    x1,x2,y1,y2 = np.min(lon)-2, np.max(lon)+2, np.min(lat)-2, np.max(lat)+2

    axins = ax.inset_axes(
        [0.5, 0.5, 0.47, 0.47],
        xlim=(x1, x2), ylim=(y1, y2), xticklabels=[], yticklabels=[],projection=ccrs.PlateCarree())

    axins.coastlines()
    axins.add_feature(cfeature.LAND)
    axins.add_feature(cfeature.OCEAN)
    axins.gridlines(draw_labels=False)
    axins.set_extent([x1,x2,y1,y2], crs=ccrs.PlateCarree())
    axins.plot(eddy_sel.longitude, eddy_sel.latitude,color='yellow', label='Trayectoria del centroide', transform=ccrs.PlateCarree())
    axins.plot(lon_last, lat_last, marker='o', markersize=2, linestyle='-', color='k',alpha=0.5,
                label=f'Contorno final', transform=ccrs.PlateCarree())
    axins.plot(lon_center_last, lat_center_last, marker='x', markersize=10, color='k', alpha=0.5,
                label=f'Centro final', transform=ccrs.PlateCarree())
    axins.plot(lon, lat, marker='o', markersize=2, linestyle='-', color=col,
                label=f'Contorno inicial', transform=ccrs.PlateCarree())
    axins.plot(lon_center, lat_center, marker='x', markersize=10, color=col, alpha=0.5,
                label=f'Centro inicial', transform=ccrs.PlateCarree())
    ax.set_title(f"Remolino {eddy_type_str} {eddy_num} - {eddy_sel_time_ini.strftime('%d-%b-%Y')} al {eddy_sel_time_end.strftime('%d-%b-%Y')}")

    ax.indicate_inset_zoom(axins, edgecolor="black", linewidth=1)
    fig_ref['fig'] = fig
    plt.show()
    display(save_map_button)
    
    
def load_argo_profiles(lat, lon, eddy_sel_time_ini, eddy_sel_time_end, buff=0.25):
    
    ini_date = (eddy_sel_time_ini-pd.DateOffset(days=15)).strftime('%Y-%m')
    fin_date = (eddy_sel_time_end+pd.DateOffset(days=15)).strftime('%Y-%m')
    
    dfetch = argo.DataFetcher()
    dfetch = dfetch.region([np.min(lon)-buff, np.max(lon)+buff, 
                            np.min(lat)-buff, np.max(lat)+buff, 
                            0, 1000, ini_date, fin_date])
    try:
        ds = dfetch.load().data
        ds_profiles = ds.argo.point2profile()
        print(f"Hay {len(ds_profiles.N_PROF.values)} perfiles Argo en la región del remolino")
        return ds_profiles
    except FileNotFoundError:
        print("No hay perfiles Argo en la región del remolino, intenta otro número de remolino")
        return None

def plot_argo_profiles(ds_profiles, lat, lon, lat_center, lon_center, eddy_sel_time_ini, eddy_sel_time_end, eddy_type_str, eddy_num, lon_center_last, lat_center_last, col='blue'):
    if ds_profiles is None:
        print("No hay perfiles Argo para mostrar.")
        return 
    fig2 = plt.figure(figsize=(18, 8))
    ax_map = plt.subplot(1, 3, 1, projection=ccrs.PlateCarree())
    ax_prof_t = plt.subplot(1, 3, 2)
    ax_prof_s = plt.subplot(1, 3, 3)

    ax_map.coastlines()
    ax_map.set_extent([lon_center-2, lon_center+2, lat_center-2, lat_center+2], crs=ccrs.PlateCarree())
    ax_map.add_feature(cfeature.LAND)
    ax_map.add_feature(cfeature.OCEAN)
    ax_map.gridlines(draw_labels=True)
    ax_map.plot(eddy_sel.longitude, eddy_sel.latitude,color='yellow')
    ax_map.plot(lon_last, lat_last, marker='o', markersize=2, linestyle='-', color='k',alpha=0.5,
                label=f'Contorno final', transform=ccrs.PlateCarree())
    ax_map.plot(lon_center_last, lat_center_last, marker='x', markersize=10, color='k', alpha=0.5,
                label=f'Centro final', transform=ccrs.PlateCarree())
    ax_map.plot(lon, lat, marker='o', markersize=2, linestyle='-', color=col,
                label=f'Contorno inicial', transform=ccrs.PlateCarree())
    ax_map.plot(lon_center, lat_center, marker='x', markersize=10, color=col, alpha=0.5,
                label=f'Centro inicial', transform=ccrs.PlateCarree())
    ax_map.set_title(f"Remolino {eddy_type_str} {eddy_num} - {eddy_sel_time_ini.strftime('%d-%b-%Y')} al {eddy_sel_time_end.strftime('%d-%b-%Y')}")
    
    for ii in range(len(ds_profiles.N_PROF.values)):
        LAT = ds_profiles.sel(N_PROF=ii).LATITUDE.data
        LON = ds_profiles.sel(N_PROF=ii).LONGITUDE.data
        PRES = ds_profiles.sel(N_PROF=ii).PRES.data
        TEMP = ds_profiles.sel(N_PROF=ii).TEMP.data
        PSAL = ds_profiles.sel(N_PROF=ii).PSAL.data
        TIME = ds_profiles.sel(N_PROF=ii).TIME
        prof_time = pd.to_datetime(TIME.values)
        ax_prof_t.plot(TEMP, PRES, label=f'{prof_time.strftime("%d-%b-%Y")}')
        ax_prof_s.plot(PSAL, PRES, label=f'{prof_time.strftime("%d-%b-%Y")}')
        ax_map.plot(LON, LAT, marker='^', markersize=10, markeredgecolor='k')

    ax_prof_t.invert_yaxis()
    ax_prof_t.set_xlabel('Temperatura (°C)')
    ax_prof_t.set_ylabel('Presión (dbar)')
    ax_prof_t.set_title('Perfiles Argo cercanos al remolino')
    ax_prof_t.legend()
    ax_prof_s.invert_yaxis()
    ax_prof_s.set_xlabel('Salinidad práctica (PSU)')
    ax_prof_s.legend()
    plt.tight_layout()
    fig_ref2['fig2'] = fig2
    plt.show()
    display(save_profiles_button)

# Widget callbacks and setup
def on_plot_map_clicked(b):
    #clear_output(wait=True)
    eddy_num = eddy_num_widget.value
    with outmap:
        outmap.clear_output()
        plot_eddy_map(lat, lon, lat_center, lon_center, eddy_time, eddy_sel,
                      eddy_type_str, eddy_num, eddy_sel_time_ini, eddy_sel_time_end)
    
def on_plot_profiles_clicked(b):
    eddy_num = eddy_num_widget.value
    ds_profiles = load_argo_profiles(lat, lon, eddy_sel_time_ini, eddy_sel_time_end, buff=0.25)
    with outprofiles:
        outprofiles.clear_output()
        if ds_profiles is not None:
            print(f"Perfiles Argo cercanos al remolino {eddy_type_str} {eddy_num} ({lat_center}, {lon_center})")
            plot_argo_profiles(ds_profiles, lat, lon, lat_center, lon_center, eddy_sel_time_ini, eddy_sel_time_end, eddy_type_str, eddy_num, lon_center_last, lat_center_last, col='blue')

        else:
            print("No se encontraron perfiles Argo cercanos al remolino. Intenta otro número de remolino.")
        
def on_load_data_clicked(b):
    global lat, lon, lat_center, lon_center, eddy_time, eddy_radius, eddy_sel_time_ini, eddy_sel_time_end, eddy_sel, lat_last, lon_last, lat_center_last, lon_center_last, eddy_type_str
    #clear_output(wait=True)
    display(eddy_type_widget, eddy_num_widget, load_data_button)
    eddy_type = eddy_type_widget.value
    eddy_num = eddy_num_widget.value
    try:
        eddy_ds = load_eddy_data(eddy_type)
        lat, lon, lat_center, lon_center, eddy_time, eddy_radius, eddy_sel_time_ini, eddy_sel_time_end, eddy_sel, lat_last, lon_last, lat_center_last, lon_center_last = extract_eddy(eddy_ds, eddy_num)
        eddy_type_str = 'ciclónico' if eddy_type == 'C' else 'anticiclónico'
        col = 'blue' if eddy_type == 'C' else 'red'
        eddy_time = pd.to_datetime(eddy_ds.time[eddy_num].values)
        with output:
            output.clear_output()
            print(f"Centro del remolino {eddy_num}: lat {lat_center}, lon {lon_center})")
            print(f"Fecha de inicio del remolino {eddy_sel_time_ini.strftime('%d-%b-%Y')} y fecha de fin {eddy_sel_time_end.strftime('%d-%b-%Y')}")
            print(f"Radio del remolino {eddy_num}: {eddy_radius/1e3} km") 
            display(plot_map_button, outmap, plot_profiles_button, outprofiles)
    except ValueError as e:
        print(e)
    
load_data_button = widgets.Button(description='Cargar datos de remolino', button_style='primary')
plot_map_button = widgets.Button(description='Graficar en mapa', button_style='success')
plot_profiles_button = widgets.Button(description='Graficar perfiles Argo', button_style='info')

eddy_type_widget = widgets.Dropdown(
    options=[('Ciclónico (C)', 'C'), ('Anticiclónico (A)', 'A')],
    value='C',
    description='Tipo de remolino:',
    style={'description_width': 'initial'}
)

eddy_num_widget = widgets.BoundedIntText(
    value=1,
    min=1,
    max=26000,
    step=1,
    description='Número de remolino:',
    style={'description_width': 'initial'}
)

def save_figure(b):
    fig = fig_ref['fig']
    if fig is not None:
        filename = f'mapa_remolino_{eddy_type_str}_{eddy_num_widget.value}.png'
        fig.savefig(filename, bbox_inches='tight')
        with output:
            print(f"Mapa guardado como {filename}")
    else:
        with output:
            print("No hay mapa para salvar, por favor genera el mapa primero.")


def save_profiles(b):
    fig2 = fig_ref2['fig2']
    if fig2 is not None:
        filename = f'perfiles_remolino_{eddy_type_str}_{eddy_num_widget.value}.png'
        fig2.savefig(filename, bbox_inches='tight')
        with output:
            print(f"Perfiles guardados como {filename}")
    else:
        with output:
            print("No hay perfiles para salvar, por favor genera las gráficas primero.")


output = widgets.Output()
outmap = widgets.Output()
outprofiles = widgets.Output()
save_map_button = widgets.Button(description="Guardar mapa")
save_profiles_button = widgets.Button(description="Guardar perfiles")

# Initialize globals
global lat, lon, lat_center, lon_center, eddy_time, eddy_radius, eddy_type_str, lat_last, lon_last, lat_center_last, lon_center_last, eddy_sel_time_ini, eddy_sel_time_end
lat = lon = lat_center = lon_center = eddy_time = eddy_radius = eddy_type_str = lat_last = lon_last = lat_center_last = lon_center_last = eddy_sel_time_ini = eddy_sel_time_end = None
fig_ref = {'fig': None}
fig_ref2 = {'fig2': None}
# Connect buttons
load_data_button.on_click(on_load_data_clicked)
plot_map_button.on_click(on_plot_map_clicked)
plot_profiles_button.on_click(on_plot_profiles_clicked)
save_map_button.on_click(save_figure)
save_profiles_button.on_click(save_profiles)


# Initial display
display(eddy_type_widget, eddy_num_widget, load_data_button, output)

Dropdown(description='Tipo de remolino:', options=(('Ciclónico (C)', 'C'), ('Anticiclónico (A)', 'A')), style=…

BoundedIntText(value=1, description='Número de remolino:', max=60000, min=1, style=DescriptionStyle(descriptio…

Button(button_style='primary', description='Cargar datos de remolino', style=ButtonStyle())

Output()